# Introduccio a a NLP amb Pytorch

**Assignatura**: Models d'intel·ligència artificial

**Professor** : Ramon Mateo Navarro

En aquest notebook aprendrem algunes de les bases del NLP. En el següent encara profunditzarem més en aquesta tasca.

Aquest tutorial està basat en el de Microsoft learns el qual s'ha adaptat per aquest curs i els vostres coneixements. Podeu trobar el tutorial en aquest [Link](https://learn.microsoft.com/es-es/training/modules/intro-natural-language-processing-pytorch/2-represent-text-as-tensors)

## Requisits

Primer de tot instal·larem les llibreries essencials per poder fer això. En el següent .txt s'han recopilat les més importants.

In [ ]:
%pip install torchtext
%pip install -r https://raw.githubusercontent.com/MicrosoftDocs/pytorchfundamentals/main/nlp-pytorch/requirements.txt

In [ ]:
pip install torchdata

In [ ]:
%pip install portalocker>=2.0.0

En aquest notebook, començarem amb una senzilla tasca de classificació de text basada en el conjunt de dades de mostra ***AG.NEWS***, que consisteix a classificar els titulars de notícies en una de les 4 categories: .World, Sports, Business i Sci/Tech... Aquest conjunt de dades es construeix a partir del mòdul ``torchtext``. de ``PyTorch``, de manera que podem accedir-hi fàcilment.


In [ ]:
import torch
import torchtext
import os
import collections
torch.utils.data.datapipes.utils.common.DILL_AVAILABLE = torch.utils._import_utils.dill_available()
import torchdata
import torchtext.datasets
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

Podeu observar com hem creat un directori que es dirà data on guardarem allà el dataset. Després hem creat una llista on tindrem els .

Aquí, trainsetdataset i test_dataset contenen iterators que retornen parells d'etiqueta (nombre de classe) i text respectivament, per exemple:


In [ ]:
next(train_dataset)

o iterant...

In [ ]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}\n")

Així que com ara el volem fer servir diverses vegades per tal de fer-ho més fàcil ho transformarem en una llista

In [ ]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## Tokenització i vectorització

Ara necessitem convertir text en **números** que es puguin representar com a tensors per alimentar-los en una xarxa neuronal. El primer pas és convertir text a tokens - *tokenization**. Si utilitzem la representació a nivell de paraula, cada paraula estaria representada pel seu propi token. Utilitzarem el tokenizer integrat des del mòdul ``torchtext``

In [ ]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')

Utilitzarem el tokenizer de PyTorch per dividir paraules i espais en els primers 2 articles de notícies. En el nostre cas, utilitzem BASIC forenglish per al tokenizer per entendre l'estructura del llenguatge. Això retornarà una llista de cadenes del text i els caràcters.


In [ ]:
first_sentence = train_dataset[0][1]
second_sentence = train_dataset[1][1]

f_tokens = tokenizer(first_sentence)
s_tokens = tokenizer(second_sentence)

print(f'\nfirst token list:\n{f_tokens}')
print(f'\nsecond token list:\n{s_tokens}')

## Word embeddings

Abans de proseguir fem una petita pausa i anem a entendre un concepte que està molt relacionat aqui, els **word embeddings**. 

Els "word embeddings" o incrustacions de paraules són una tècnica utilitzada en el processament del llenguatge natural per representar les paraules en forma de vectors de números. La idea bàsica és transformar cada paraula en un vector dens que captura informació sobre el seu significat i el context en què sol aparèixer. Aquests vectors són creats de manera que les paraules que tenen significats similars o que s'utilitzen en contextos similars estiguin representades per vectors que estan propers entre si en l'espai vectorial.

**Com funcionen?** 

Quan es crea un "word embedding", el model aprendrà a assignar vectors a les paraules de manera que la distància entre els vectors reflecteixi les relacions semàntiques i sintàctiques entre les paraules. Això s'aconsegueix mitjançant el entrenament en un gran corpus de text, on el model utilitza el context d'una paraula (les paraules que l'envolten) per predir la paraula mateixa.

**Per què són útils?**
* **Reducció de la Dimensionalitat**: Convertir paraules en vectors compactes redueix la complexitat i facilita el maneig computacional.
* **Similitud Semàntica**: Permet als models de NLP detectar similituds semàntiques entre paraules, millorant així la qualitat de tasques com la cerca de text, la classificació de text i més.
* **Flexibilitat**: Poden ser utilitzats en una varietat d'aplicacions de NLP, des de sistemes de resposta automàtica fins a anàlisi de sentiments.

![](images_lab_nlp\word_embeddings_image.png)

A continuació, per convertir text a números, necessitarem construir un vocabulari de tots els tokens. Primer construïm el diccionari utilitzant l'objecte ``Counter`` i després creem un objecte ``Vocab`` que ens ajudaria a fer front a la vectorització:


In [ ]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.Vocab(counter, min_freq=1)

Per veure com cada paraula s'assigna al vocabulari, farem un bucle a través de cada paraula de la llista per cercar el seu número d'índex en ``vocab``. Cada paraula o caràcter es mostra amb l'índex corresponent. Per exemple, la paraula ``the`` apareix diverses vegades en ambdues frases i és un índex únic en el vocab és el nombre **3**.


In [ ]:
word_lookup = [list((vocab[w], w)) for w in f_tokens]
print(f'\nIndex lockup in 1st sentence:\n{word_lookup}')

word_lookup = [list((vocab[w], w)) for w in s_tokens]
print(f'\nIndex lockup in 2nd sentence:\n{word_lookup}')

Utilitzant vocabulari, podem codificar fàcilment la nostra cadena tokenitzada en un conjunt de nombres. Fem servir el primer article de notícies com a exemple:


In [ ]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

def encode(x):
    return [vocab.stoi[s] for s in tokenizer(x)]

vec = encode(first_sentence)
print(vec)

En aquest codi, el diccionari torchtext ``vocab.stoi``. ens permet convertir des d'una representació de cadena en números (el nom stoi* significa "des de *s**tring *a** *i**ntegers). Per a tornar a convertir el text d'una representació numèrica en text, podem utilitzar el diccionari ``vocab.itos`` per a realitzar una cerca inversa.
